Udacity Machine Learning Engineer Nanodegree Project 5 
=============

Deep Learning for Satellite Image Recognition
-------------

In [10]:
import os
import scipy.io as sio
import numpy as np

## 1. Exploring and Preparing the Data

First, we will extract and investigate the images from the SAT-6 dataset. Information about the dataset is available here: 
    
    http://csc.lsu.edu/~saikat/deepsat/ 

The dataset contains 405,000 satellite images, and is separated into 324,000 images for the training dataset and 81,000 for the testing dataset. We start by opening the mat file.

In [ ]:
def open_mat(filename):
    #opens the SAT6 dataset, which is a .mat file.
    if os.path.isfile(filename): 
        print ("Current working directory contains %r" % filename)
        mat_contents = sio.loadmat(filename)
        print ("SAT6 loaded successfully.")
    else:
        print ("Download %r to current working directory" % filename)
    return mat_contents

mat_sat_6 = 'sat-6-full.mat'

sat_6 = open_mat(mat_sat_6)

Current working directory contains 'sat-6-full.mat'


Next, we will explore the dataset.

In [ ]:
def investigate_dict(a, b):
    keys_list = a.keys()
    keys_str = ', '.join(keys_list)
    lst = []
    print("File is a %r, with keys %r." % (type(a), keys_str))
    for k in b:
        array = a.get(k)
        print('%r has dimensions %r' % (k, array.shape))
        lst.append(array)
    return lst

sets = ['train_x', 'train_y', 'test_x', 'test_y']

raw_x_train, raw_y_train, raw_x_test, raw_y_test = investigate_dict(sat_6, sets)
print(sat_6['annotations'])

There are 324,000 training images and 81,000 testing images. Each image is 28x28, meaning each picture has 784 pixels. Each image also has 4 layers (Red, Green, Blue and Near-Infared), and is labeled with a binary vector. The vector has 6 elements and each element corresponds with the six classes the image could belong to (buildings, barren land, trees, roads, buildings and water bodies). 

Let's change the dimensions of the array so that the image index is the first dimension, the type of layer is the second, and the pixels are the third. This will make it simpler to use.

In [ ]:
def change_dim(a):
    lst = []
    
    for i in a:
        
        if len(i.shape) == 4:
            i_2 = np.rollaxis(i, 3)
            i_3 = np.rollaxis(i_2, 3, start = 1)
            lst.append(i_3)
        
        elif len(i.shape) == 2:
            i_4 = np.rollaxis(i, 1)
            lst.append(i_4)
        
        else:
            print('Dimensions of arrays do not work')
    
    return lst


raw = [raw_x_train, raw_y_train, raw_x_test, raw_y_test]

switch_x_train, switch_y_train, switch_x_test, switch_y_test = change_dim(raw)

print(switch_x_train.shape)
print(switch_y_train.shape)
print(switch_x_test.shape)
print(switch_y_test.shape)

Let's view a sample of the images and their corresponding labels.

In [ ]:
def sample_images(labels, x_data, y_data, n):
    
    
    for i in range(n):
        r = np.random.choice(len(x_data))
        #Create an rgb array
        rgbArray = np.zeros((28,28,3), 'uint8')
        rgbArray[..., 0] = x_data[r,0,...]
        rgbArray[..., 1] = x_data[r,1,...]
        rgbArray[..., 2] = x_data[r,2,...]
        img = Image.fromarray(rgbArray)
        out = img.resize((48, 48))
        display(out)

        #returns the index of 1 from the label array
        label_index = y_data[r].tolist().index(1)
        print(labels[label_index])

land_cover = ['buildings', 'barren_land', 'trees', 'grassland', 'roads', 'water_bodies']

sample_images(land_cover, switch_x_train, switch_y_train, 30)

Next, we will divide our dataset into training, testing, and cross validation sets. Currently,  80% of the dataset is reserved for training, and 20% of the dataset is reserved for testing. To prevent bias, we will concatenate the training and test sets 

In [ ]:
#Combine original train and test set together.
X_all = np.concatenate((switch_x_train, switch_x_test), axis=0)
y_all = np.concatenate((switch_y_train, switch_y_test), axis = 0)

#Shuffle the data
perm = random.permutation(len(X_all))
X_all_shuffled = X_all[perm]
y_all_shuffled = y_all[perm]

#Verify that the labels and images are still correct
sample_images(land_cover, X_all_shuffled, y_all_shuffled, 5)

It seems that the image and label pairs have been maintained after shuffling. Since it would be computationally expensive to use k-fold cross validation to train a deep learning architecture with a large amount of training examples, we will create a simple, hold-out cross validation set as well as a training and test set.